# imports

In [1]:
import os, json
import openai
from dotenv import dotenv_values, load_dotenv
from openai.types.chat.chat_completion import ChatCompletion
from IPython.display import display, Markdown


# load api key

In [2]:
load_dotenv()
client:openai.OpenAI = openai.OpenAI()

# functions
## openai helper function

In [3]:
def get_llm_response(client:openai.OpenAI, prompt:str, model:str='gpt-4o-mini', return_dict_body:bool=False, return_response_body:bool=False) -> str | dict | ChatCompletion:
    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model=model,
    )
    
    # print(type(response))
    
    if return_response_body:
        return response
    if return_dict_body:
        return response.model_dump()
    return response.choices[0].message.content


## prompt template interpolate function

In [4]:
def get_colors_prompt(prompt:str) -> str:
    return f'''
    You are a color palette generating assistant that responds to text prompts for color palettes
    Your should generate color palettes that fit the theme, mood, or instructions in the prompt.
    The palettes should be between 2 and 8 colors, the more the better.

    Q: Convert the following verbal description of a color palette into a list of colors: The Mediterranean Sea
    A: ["#006699", "#66CCCC", "#F0E68C", "#008000", "#F08080"]

    Q: Convert the following verbal description of a color palette into a list of colors: sage, nature, earth
    A: ["#EDF1D6", "#9DC08B", "#609966", "#40513B"]


    Desired Format: just a JSON array of hexadecimal color codes, nothing else before or after

    Q: Convert the following verbal description of a color palette into a list of colors: {prompt}
    A:    
    '''


## notebook markdown display helper

In [5]:
def display_colors(colors, width:int=4):
    display(
        Markdown(
            ' '.join(
                f'<span style="color: {color}">{chr(9608) * width}</span>'
                for color in colors
            )
        )
    )

# main
## get json color array

In [6]:
response = get_llm_response(
    client, 
    get_colors_prompt('blue ocean')
)
response

'["#003366", "#0077be", "#00aaff", "#99ccff", "#e0f7ff"]'

## convert json to python array

In [7]:
colors = json.loads(response)
colors

['#003366', '#0077be', '#00aaff', '#99ccff', '#e0f7ff']

## display result

In [8]:
display_colors(colors)

<span style="color: #003366">████</span> <span style="color: #0077be">████</span> <span style="color: #00aaff">████</span> <span style="color: #99ccff">████</span> <span style="color: #e0f7ff">████</span>

# macro helper functions

In [9]:
def get_color_palette(client:openai.OpenAI, prompt:str) -> list:
    userprompt = get_colors_prompt(prompt)
    response = get_llm_response(client, userprompt)
    colors = json.loads(response)
    return colors


In [10]:
from json import JSONDecodeError


def show_palette(client:openai.OpenAI, prompt:str):
    try:
        colors = get_color_palette(client, prompt)
        display_colors(colors)
        print(colors)
    except JSONDecodeError as jde:
        print(f'something wrong with llm response format: {response!r}')

In [11]:
show_palette(client, 'ocean abyss')

<span style="color: #003B5C">████</span> <span style="color: #005C99">████</span> <span style="color: #008B8B">████</span> <span style="color: #4FC3F7">████</span> <span style="color: #A1C6EA">████</span> <span style="color: #E0F7FA">████</span>

['#003B5C', '#005C99', '#008B8B', '#4FC3F7', '#A1C6EA', '#E0F7FA']


In [12]:
show_palette(client, 'cyber terminal with green chars')

<span style="color: #00FF00">████</span> <span style="color: #000000">████</span> <span style="color: #1A1A1A">████</span> <span style="color: #4DFF4D">████</span> <span style="color: #007F00">████</span>

['#00FF00', '#000000', '#1A1A1A', '#4DFF4D', '#007F00']


In [13]:
show_palette(client, 'spongebob')

<span style="color: #F4D03F">████</span> <span style="color: #F39C12">████</span> <span style="color: #E74C3C">████</span> <span style="color: #2980B9">████</span> <span style="color: #2ECC71">████</span>

['#F4D03F', '#F39C12', '#E74C3C', '#2980B9', '#2ECC71']


In [14]:
show_palette(client, 'superman')

<span style="color: #C8102E">████</span> <span style="color: #003DA5">████</span> <span style="color: #FEC200">████</span> <span style="color: #FFFFFF">████</span>

['#C8102E', '#003DA5', '#FEC200', '#FFFFFF']


In [15]:
show_palette(client, 'european tabby cat white gray and black')

<span style="color: #FFFFFF">████</span> <span style="color: #A9A9A9">████</span> <span style="color: #7D7D7D">████</span> <span style="color: #000000">████</span>

['#FFFFFF', '#A9A9A9', '#7D7D7D', '#000000']
